In [61]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc,html,Input,Output

In [62]:
data = {
    "Year": [
        1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974, 1978,
        1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014, 2018, 2022
    ],
    "Winner": [
        "Uruguay", "Italy", "Italy", "Uruguay", "West Germany", "Brazil", "Brazil", "England",
        "Brazil", "West Germany", "Argentina", "Italy", "Argentina", "West Germany", "Brazil",
        "France", "Brazil", "Italy", "Spain", "Germany", "France", "Argentina"
    ],
    "Runner-Up": [
        "Argentina", "Czechoslovakia", "Hungary", "Brazil", "Hungary", "Sweden", "Czechoslovakia", "West Germany",
        "Italy", "Netherlands", "Netherlands", "West Germany", "West Germany", "Argentina", "Italy",
        "Brazil", "Germany", "France", "Netherlands", "Argentina", "Croatia", "France"
    ]
}
# This is all the  countries that have won a world cup from wikipedia
df = pd.DataFrame(data)

# convert west germany to germany
country_map = {"West Germany": "Germany"}
df.replace(country_map, inplace=True)

# Count number of wins
winnercount = df["Winner"].value_counts().reset_index()
winnercount.columns = ["Winner", "Count"]

# Dash App Setup
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("FIFA World Cup Dashboard", style={"textAlign": "center"}),

    html.Div([
        html.H2("World Cup Wins by Country (Choropleth Map)"),
        dcc.Graph(id='choropleth-map')
    ], style={"marginBottom": 50}),

    html.Div([
        html.Label("Select a country to view number of wins:"),
        dcc.Dropdown(
            id="country-dropdown",
            options=[{"label": c, "value": c} for c in sorted(winnercount["Winner"].unique())],
            value="Brazil"
        ),
        html.Div(id='country-output', style={"marginTop": 10})
    ], style={"marginBottom": 50}),

    html.Div([
        html.Label("Select a World Cup year to view winner and runner-up:"),
        dcc.Dropdown(
            id='year-dropdown',
            options=[{'label': str(y), 'value': y} for y in sorted(df['Year'])],
            value=2022
        ),
        html.Div(id='year-output', style={"marginTop": 10})
    ])
])

# Callback to update choropleth map
@app.callback(
    Output('choropleth-map', 'figure'),
    Input('country-dropdown', 'value')
)
def update_choropleth(_):
    fig = px.choropleth(
        winnercount,
        locations="Winner",
        locationmode="country names",
        color="Count",
        color_continuous_scale="Blues",
        title="Countries that have won the FIFA World Cup"
    )
    return fig

# Callback to show number of wins
@app.callback(
    Output('country-output', 'children'),
    Input('country-dropdown', 'value')
)
def update_country_output(selected_country):
    wins = winnercount[winnercount['Winner'] == selected_country]['Count'].values[0]
    return f"{selected_country} has won the FIFA World Cup {wins} time(s)."

# Callback to show winner and runner-up of a selected year
@app.callback(
    Output('year-output', 'children'),
    Input('year-dropdown', 'value')
)
def update_year_output(selected_year):
    row = df[df['Year'] == selected_year].iloc[0] # properly get the correct year and country wins
    return f"In {selected_year}, the winner was {row['Winner']} and the runner-up was {row['Runner-Up']}."

# Run app
if __name__ == '__main__':
    app.run(debug=True,port=8887)
